In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import GPy
from scipy.stats import pearsonr,spearmanr
from sklearn.decomposition import PCA
import sklearn.gaussian_process as gp
from scipy.stats import zscore
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.linear_model import LinearRegression,Ridge,ElasticNet
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import torch
import gpytorch
from gpytorch.constraints import Positive, Interval
from gpytorch.means import Mean
import math
import statsmodels.api as sm
from matplotlib.legend import Legend




import seaborn as sns
sns.set(color_codes=True)
sns_c = sns.color_palette(palette='deep')
sns.set_style('whitegrid')
sns.set_context('paper',font_scale=2)

%run Utilities.ipynb
%matplotlib inline

In [ ]:
def stratified_spatial_v2(comreg_df,map_distances,threshold):
    rnd_reg = np.random.choice(comreg_df['REG'].unique())
    rnd_com = comreg_df.loc[comreg_df['REG'] == rnd_reg]['CCOD_CRCA'].sample(1).iloc[0]
    f = 50 # 1 km = 0.00895305 degrees
    data_partition = pd.DataFrame(comreg_df[['CCOD_CRCA','row_index']].copy(),index=comreg_df.index)
    data_partition['type'] = 'train'
    
    training_inds = []
    # starting from commune, add all other communes that are within rad to training and rest 
    #to test
    while len(training_inds) < threshold:
        f = f + 50
        training_inds = [rnd_com]
        testing_inds = []
        rad = f * 0.00895305
        training_inds = training_inds + list(map_distances.loc[
            (map_distances['InputID'] == rnd_com) & (map_distances['Distance'] < rad)]['TargetID'])
    testing_inds = list(set(list(comreg_df['CCOD_CRCA'])).difference(set(training_inds)))
    data_partition.loc[data_partition['CCOD_CRCA'].isin(testing_inds),'type'] = 'test'

    return data_partition, rnd_reg, rnd_com

In [ ]:
def crossvalidate_spatial(x,y,alpha,niters,classifier,args,map_distances,comreg_df):
    selected_coms = []
    selected_regs = []
    #map to store the actual predictions
    map_YPreds = []
    map_YVar1 = []
    map_YVar2 = []
    map_YVar_final = []
    
    rmsevec = []
    rmseurbanvec = []
    rmseruralvec = []
    rmseurbanvec_d = []
    rmseurbanvec_nd = []
    pcorrvec = []
    scorrvec = []
    ppvalvec = []
    spvalvec = []

    ny = y.shape[1]
    s = args['s']
    urtype = args['urtype'] # urban rural type
   
    ii = 0
    while ii < niters:
        # split data for crossvalidation
        threshold = 225
        #threshold = 330
        data_partition,rnd_reg,rnd_com = stratified_spatial_v2(comreg_df,map_distances,threshold)
        if ii % 10 == 0 and ii > 0:
            print("Validation run %d of %d"%(ii,niters))
        ii = ii + 1
        train_index = list(data_partition.loc[data_partition['type'] == 'train','row_index'].values)
        test_index = list(data_partition.loc[data_partition['type'] == 'test','row_index'].values)
        test_regions = data_partition.loc[data_partition['type'] == 'test','CCOD_CRCA'].str[0:2].astype(int).values
        XTrain, XTest, STrain, STest = x[train_index,:], x[test_index,:], s[train_index,:], s[test_index,:]
        urTrain, urTest = urtype[train_index,:],urtype[test_index,:]
        YTrain, YTest = y[train_index,:], y[test_index,:]
        if classifier in ['MultiViewGPRSpatial','Mixed' ,'Kriging','GPRSpatial']:
            args['STrain'] = STrain
            args['STest'] = STest
        if classifier in ['GPRSpatial']:
            args['urTrain'] = urTrain
            args['urTest'] = urTest
        try:
            YPred,Vars = evaluate(XTrain,YTrain,XTest,alpha,classifier,args)
            map_YPreds.append((test_index,YPred))
            map_YVars = {}
            if 'YVar' in Vars.keys():
                YVar = Vars['YVar']
                map_YVar_final.append((test_index,YVar))
            if 'Var1' in Vars.keys():
                YVar1 = Vars['Var1']
                map_YVar1.append((test_index,YVar1))
            if 'Var2' in Vars.keys():
                YVar2 = Vars['Var2']
                map_YVar2.append((test_index,YVar2))
            selected_coms.append(rnd_com)
            selected_regs.append(rnd_reg)
        except NameError:
            print("error identified")
            continue

        rmse = np.zeros([ny,])
        rmse[0:ny] = np.sqrt(np.mean((YPred[:,0:ny] - YTest)**2,axis=0))
        rmserural = np.zeros([ny,])
        rmseurban = np.zeros([ny,])
        rmseurban_d = np.zeros([ny,]) # dakar
        rmseurban_nd = np.zeros([ny,])# non-dakar

        urbinds = np.where(urTest == 1)[0]
        rurinds = np.where(urTest == 0)[0]
        urb_dinds = np.array(set(urbinds).intersection(set(np.where(test_regions==1)[0])))
        urb_ndinds = np.array(set(urbinds).intersection(set(np.where(test_regions!=1)[0])))
  
        rmseurban = np.sqrt(np.mean((YPred[urbinds,0:ny] - YTest[urbinds,:])**2,axis=0))
        rmserural = np.sqrt(np.mean((YPred[rurinds,0:ny] - YTest[rurinds,:])**2,axis=0))
        rmseurban_d = np.sqrt(np.mean((YPred[urb_dinds,0:ny] - YTest[urb_dinds,:])**2,axis=0))
        rmseurban_nd = np.sqrt(np.mean((YPred[urb_ndinds,0:ny] - YTest[urb_ndinds,:])**2,axis=0))


        pcorr = np.zeros([ny,])
        scorr = np.zeros([ny,])
        ppval = np.zeros([ny,])
        spval = np.zeros([ny,])

        for i in range(ny):
            pcorr[i] = pearsonr(YPred[:,i],YTest[:,i])[0]
            ppval[i] = pearsonr(YPred[:,i],YTest[:,i])[1]
            scorr[i] = spearmanr(YPred[:,i],YTest[:,i])[0]
            spval[i] = spearmanr(YPred[:,i],YTest[:,i])[1]
        
        pcorrvec.append(pcorr)
        ppvalvec.append(ppval)
        scorrvec.append(scorr)
        spvalvec.append(spval)
        rmsevec.append(rmse)
        rmseurbanvec.append(rmseurban)
        rmseruralvec.append(rmserural)
        rmseurban_dvec.append(rmseurban_d)
        rmseurban_ndvec.append(rmseurban_nd)


    rmsevec = np.array(rmsevec)
    rmseurbanvec = np.array(rmseurbanvec)
    rmseruralvec = np.array(rmseruralvec)
    rmseurban_dvec = np.array(rmseurban_dvec)
    rmseurban_ndvec = np.array(rmseurban_ndvec)
    pcorrvec = np.array(pcorrvec)
    ppvalvec = np.array(ppvalvec)
    scorrvec = np.array(scorrvec)
    spvalvec = np.array(spvalvec)

    return pcorrvec,ppvalvec,scorrvec,spvalvec,rmsevec,map_YPreds,map_YVar1,map_YVar2,map_YVar_final,rmseurbanvec,rmseruralvec,rmseurban_dvec,rmseurban_ndvec

In [ ]:
def evaluate(XTrain,YTrain,XTest,alpha,classifier,args):
    supported_methods = ['GPR','GPRSpatial','LinearRegression']
    if classifier not in supported_methods:
        print('Only following classifiers are supported')
        print(supported_methods)
        raise ValueError('Undefined classifier specified')
        
    Vars = {}
    # learn model
    if classifier == 'LinearRegression':
        lmodel = LinearRegression()
        lmodel.fit(XTrain,YTrain)
        YPred = lmodel.predict(XTest)
    elif classifier == 'GPR':
        YPred,var = run_GP(XTrain,YTrain,XTest,alpha,[],[])
        Vars['YVar'] = var
    elif classifier == 'GPRSpatial':
        STrain = args['STrain']
        STest = args['STest']
        urTrain = args['urTrain']
        urTest = args['urTest']
        if args['mokernel'] == True:
            YPred,var = run_GPyTorchMO(XTrain,YTrain,XTest,alpha,STrain,STest,urTrain,urTest,
                                       fitlinear=args['fitlinear'],
                                       ard=args['ard'],mo_ard=args['mo_ard'],
                                       initPhis=args['initPhis'])
        else:
            YPred,var = run_GPyTorch(XTrain,YTrain,XTest,alpha,STrain,STest,urTrain,urTest,
                                     fitlinear=args['fitlinear'],ard=args['ard'])
        Vars['YVar'] = var
    else:
        raise ValueError('Undefined model specified')
    return YPred,Vars


In [ ]:
def pretty_print(a):
    st = ''
    for _n in a:
        st = st+'%05.2f '%_n
    print(st[:-1])


In [ ]:
def fillMissingCommunes(current_predictions,all_commune_ids):
    # find regional predictions
    reg_predictions = {}
    for c in current_predictions.keys():
        r = c[0:2]
        if r not in reg_predictions.keys():
            reg_predictions[r] = []
        reg_predictions[r].append(current_predictions[c])
    avg_reg_predictions = {}
    for r in reg_predictions.keys():
        avg_reg_predictions[r] = np.mean(np.array(reg_predictions[r]),axis=0)
    new_predictions = {}
    for c in all_commune_ids:
        if c in current_predictions.keys():
            new_predictions[c] = current_predictions[c]
        else:
            new_predictions[c] = avg_reg_predictions[c[0:2]]
    return new_predictions

In [ ]:
def getPredStats(map_YPreds,map_Yvar_combined,comreg_df,commune_ids):
    ## Combining predicted means for each run to get average predictions for each commune
    map_ComIndPreds = {}
    for _l  in range(len(map_YPreds)):
        #_l gives the index of the crossvalidation run
        for _j in range(len(map_YPreds[_l][0])):
            #_j gives the index of a commune
            _com_id = comreg_df.loc[comreg_df['row_index'] == map_YPreds[_l][0][_j]]['CCOD_CRCA'].values[0]
            if _com_id not in map_ComIndPreds.keys():
                map_ComIndPreds[_com_id] = []
            map_ComIndPreds[_com_id].append(map_YPreds[_l][1][_j])

    map_ComIndMeanPreds = {}
    for c in map_ComIndPreds.keys():
        v = np.array(map_ComIndPreds[c])
        map_ComIndMeanPreds[c] = np.mean(v,axis=0)
    commune_ids = list(comreg_df['CCOD_CRCA'])
    map_ComIndMeanPreds_filled = fillMissingCommunes(map_ComIndMeanPreds,commune_ids)
    predictedMeans = pd.DataFrame.from_dict(map_ComIndMeanPreds,orient='index',columns=outputs)


    if len(map_Yvar_combined) > 0:
        ## Combining predicted Variances for each run to get average variance prediction for each commune
        map_ComIndVars = {}
        for _l  in range(len(map_Yvar_combined)):
            #_l gives the index of the crossvalidation run
            for _j in range(len(map_Yvar_combined[_l][0])):
                #_j gives the index of a commune
                _com_id = comreg_df.loc[comreg_df['row_index'] == map_YPreds[_l][0][_j]]['CCOD_CRCA'].values[0]
                if _com_id not in map_ComIndVars.keys():
                    map_ComIndVars[_com_id] = []
                map_ComIndVars[_com_id].append(map_Yvar_combined[_l][1][_j])

        map_ComIndMeanVars = {}
        for c in map_ComIndVars.keys():
            v = np.array(map_ComIndVars[c])
            map_ComIndMeanVars[c] = np.mean(v,axis=0)
    
        map_ComIndMeanVars_filled = fillMissingCommunes(map_ComIndMeanVars,commune_ids)

        predictedVars = pd.DataFrame.from_dict(map_ComIndMeanVars,orient='index',columns=outputs)
    else:
        predictedVars = pd.DataFrame.from_dict({})
    return predictedMeans,predictedVars

In [ ]:
def loadLandsatData(year,resnetmodeltype='B'):
    ## load landsat data
    landsat = pickle.load(open('./landsatfeatures_{}_{}.pickle'.format(year,resnetmodeltype),'rb'))
    landsat_means = {}
    for c in landsat.keys():
        landsat_means[c] = landsat[c][0]
    landsat = pd.DataFrame.from_dict(landsat_means,orient='index')
    return landsat

In [ ]:
def loadFeaturesRaw(year,resnetmodeltype='B'):
    ## load nightlight and aod data
    df_nl_aod = pd.read_csv('nl_aod_{}_features.csv'.format(year))
    df_nl_aod['Unnamed: 0'] = df_nl_aod['Unnamed: 0'].astype(str).str.zfill(8)
    df_nl_aod.set_index('Unnamed: 0',inplace=True)
    df_nl_aod = df_nl_aod[['nl_{}_avg'.format(year),
                           'aod_{}_avg'.format(year),
                           'nl_{}_var'.format(year),
                           'aod_{}_var'.format(year),
                           'pop{}'.format(year)]]

    ## load landsat data
    landsat = pickle.load(open('./landsatfeatures_{}_{}.pickle'.format(year,resnetmodeltype),'rb'))
    landsat_means = {}
    landsat_vars = {}
    for c in landsat.keys():
        landsat_means[c] = landsat[c][0]
        landsat_vars[c] = landsat[c][1]
    landsat = pd.DataFrame.from_dict(landsat_means,orient='index')
    landsat_vars = pd.DataFrame.from_dict(landsat_vars,orient='index')
    landsat = landsat.join(landsat_vars,rsuffix='_var')
    # join everything
    data_aug = landsat.join(df_nl_aod,how='left')
    return data_aug

In [ ]:
def loadAllData(year,pcamd,resnetmodeltype='B'):
    
    ## load nightlight and aod data
    df_nl_aod = pd.read_csv('nl_aod_{}_features.csv'.format(year))
    df_nl_aod['Unnamed: 0'] = df_nl_aod['Unnamed: 0'].astype(str).str.zfill(8)
    df_nl_aod.set_index('Unnamed: 0',inplace=True)
    df_nl_aod = df_nl_aod[['nl_{}_avg'.format(year),
                           'aod_{}_avg'.format(year),
                           'nl_{}_var'.format(year),
                           'aod_{}_var'.format(year),
                           'pop{}'.format(year)]]

    ## load landsat data
    landsat = pickle.load(open('./landsatfeatures_{}_{}.pickle'.format(year,resnetmodeltype),'rb'))
    landsat_means = {}
    landsat_vars = {}
    for c in landsat.keys():
        landsat_means[c] = landsat[c][0]
        landsat_vars[c] = landsat[c][1]
    landsat = pd.DataFrame.from_dict(landsat_means,orient='index')
    landsat_vars = pd.DataFrame.from_dict(landsat_vars,orient='index')

    
    ## use pca to embed into lower dimensional space
    ls_cols = ['ls_{}'.format(i+1) for i in np.arange(pcamd.n_components)]
    landsat_sub = pd.DataFrame(pcamd.fit_transform(landsat.values),
                           index=landsat.index,columns=ls_cols)
    
    # calculate variance for each PCA feature
    ls_var_cols = []
    for i in range(n_components):
        ls_var_cols.append('ls_var_'+str(i+1))
    landsat_sub_vars = pd.DataFrame(np.dot(landsat_vars,np.power(pcamd.components_,2).T),
                                    index=landsat.index,columns=ls_var_cols)
    landsat_sub = landsat_sub.join(landsat_sub_vars)
    
    # load urban rural mapping
    urbrur = pd.read_csv('./communes_urban_rural.csv')
    urbrur['CCOD_CRCA'] = urbrur['CCOD_CRCA'].astype(str).str.zfill(8)
    urbrur.set_index('CCOD_CRCA',inplace=True)
    urbrur = pd.DataFrame(urbrur['TYPE'])

    ccoords = pd.read_csv('./urban_rural_points.csv',
                                      index_col='CCOD_CRCA')
    ccoords.index = ccoords.index.astype(str).str.zfill(8)
    
    # join everything
    data_aug = landsat_sub.join(df_nl_aod,
                            how='left').join(ccoords,
                                             how='left').join(urbrur,
                                                              how='left')
    return data_aug

In [ ]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood,ard=False,feat_kernel_type='rbf',hasTime=False):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        
        if hasTime:
            n_feats = train_x.shape[1] - 4
        else:
            n_feats = train_x.shape[1] - 3
        if ard:
            ard_num_dims = n_feats
        else:
            ard_num_dims = None
        #self.mean_module = gpytorch.means.ConstantMean()
        self.mean_module = gpytorch.means.ZeroMean()
        #self.mean_module = LinearMeanBasis(n_feats)
        if feat_kernel_type == 'matern':
            kern_feat = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(
                nu=1.5,ard_num_dims=ard_num_dims,active_dims=tuple(range(n_feats))))
        else:
            kern_feat = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(
                ard_num_dims=ard_num_dims,active_dims=tuple(range(n_feats))))
        curr_ind = n_feats
        if hasTime:
            kern_time = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(active_dims=tuple([curr_ind])))
            curr_ind += 1
        kern_spat = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(active_dims=tuple([curr_ind,curr_ind+1])))
        kern_ur = gpytorch.kernels.RBFKernel(active_dims=tuple([curr_ind+2]))
        self.likelihood.noise_covar.noise = 1
        # initialize hyper-parameters
        kern_feat.outputscale = 1
        kern_feat.base_kernel.lengthscale = 0.25
        kern_spat.outputscale = 1
        kern_spat.base_kernel.lengthscale = 0.25
        kern_ur.lengthscale = 1e-6
        if hasTime:
            kern_time.outputscale = 1
            kern_time.base_kernel.lengthscale = 0.25
        # do not optimize the urban-rural kernel's lengthscale
        kern_ur.raw_lengthscale.requires_grad_(False)
        if hasTime:
            self.covar_module = kern_feat + (kern_spat * kern_ur) + kern_time
        else:
            self.covar_module = kern_feat + (kern_spat * kern_ur)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
# kernel for energy work
class ExactGPMOModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood,num_tasks,
                 ard=False,mo_ard=False,feat_kernel_type='rbf',initPhis=None,hasTime=False):
        super(ExactGPMOModel, self).__init__(train_x, train_y, likelihood)
        
        if hasTime:
            n_feats = train_x.shape[1] - 5
        else:
            n_feats = train_x.shape[1] - 4
        
        if ard:
            ard_num_dims = n_feats
        else:
            ard_num_dims = None
            
        self.mean_module = gpytorch.means.ConstantMean()
        if feat_kernel_type == 'matern':
            kern_feat = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(
                nu=1.5,ard_num_dims=ard_num_dims,active_dims=tuple(range(n_feats))))
        else:
            kern_feat = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(
                ard_num_dims=ard_num_dims,active_dims=tuple(range(n_feats))))
        curr_ind = n_feats
        if hasTime:
            kern_time = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(active_dims=tuple([curr_ind])))
            curr_ind += 1
        kern_spat = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(active_dims=tuple([curr_ind,curr_ind+1])))
        kern_ur = gpytorch.kernels.RBFKernel(active_dims=tuple([curr_ind+2]))
        kern_mo = MOKernel(num_tasks=num_tasks,active_dims=tuple([curr_ind+3]),ard=mo_ard)

        self.likelihood.noise_covar.noise = 1
        # initialize hyper-parameters
        kern_feat.outputscale = 1
        kern_feat.base_kernel.lengthscale = 0.25
        kern_spat.outputscale = 1
        kern_spat.base_kernel.lengthscale = 0.25
        if hasTime:
            kern_time.outputscale = 1
            kern_time.base_kernel.lengthscale = 0.25
        kern_ur.lengthscale = 1e-6
        if initPhis != None:
            kern_mo.phis = initPhis
        
        # do not optimize the urban-rural kernel's lengthscale
        kern_ur.raw_lengthscale.requires_grad_(False)
        if hasTime:
            self.covar_module = (kern_feat + (kern_spat * kern_ur) + kern_time)*kern_mo # MAIN temporal kernel
        else:
            self.covar_module = (kern_feat + (kern_spat * kern_ur))*kern_mo # MAIN kernel no time

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
def run_GPyTorch(xtrain,ytrain,xtest,alpha,strain=[],stest=[],urtrain=[],urtest=[],
           fitlinear=False,return_models=False,ard=False,hasTime=False,missingvalue=-999):
    res_train = []
    if fitlinear:
        # fit linear model
        linearmodel = []
        y_hat_test = []
        for oindex in range(ytrain.shape[1]):
            lm = ElasticNet(alpha = alpha)
            minds = np.where(ytrain[:,oindex]!=missingvalue)[0]
            xt = xtrain[minds,:]
            yt = ytrain[minds,oindex:oindex+1]
            lm.fit(xt,yt)
            # get predictions for training data and test data using the linear model
            y_h_train = lm.predict(xt)
            y_h_test = lm.predict(xtest)
            linearmodel.append(lm)
            res_train.append(yt.flatten() - y_h_train)
            y_hat_test.append(y_h_test)
        y_hat_test = np.vstack(y_hat_test).T
    else:
        for oindex in range(ytrain.shape[1]):
            minds = np.where(ytrain[:,oindex]!=missingvalue)[0]
            res_train.append(ytrain[minds,oindex])
    ystar_test = np.zeros([xtest.shape[0],ytrain.shape[1]])
    ystar_var = np.zeros([xtest.shape[0],ytrain.shape[1]])
    if return_models:
        models = []
    totaltrainingsize = 0
    for oindex in range(ytrain.shape[1]):
        minds = np.where(ytrain[:,oindex]!=missingvalue)[0]
        xtrain_aug = np.hstack([xtrain[minds,:],strain[minds,:],urtrain[minds,:]])
        xtest_aug = np.hstack([xtest,stest,urtest])
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        xtrain_aug_tensor = torch.tensor(np.array(xtrain_aug,dtype='float32'))
        ytrain_tensor = torch.tensor(np.array(res_train[oindex],dtype='float32'))

        totaltrainingsize += xtrain_aug_tensor.shape[0]
        model = ExactGPModel(xtrain_aug_tensor, 
                             ytrain_tensor, 
                             likelihood, ard,hasTime=hasTime)
        train_GPyTorch(model,likelihood,xtrain_aug_tensor,ytrain_tensor,iterations=100)
        
        if return_models:
            models.append(model)
        # get predictions
        model.eval()
        likelihood.eval()
        _rstar = likelihood(model(torch.tensor(np.array(xtest_aug,dtype='float32'))))
        
        ystar_test[:,oindex] = _rstar.mean.detach().numpy().ravel()
        ystar_var[:,oindex] = _rstar.variance.detach().numpy().ravel()
    if fitlinear:
        ystar_test = y_hat_test + ystar_test
    if return_models:
        if fitlinear:
            return ystar_test,ystar_var,(models,linearmodel)
        else:
            return ystar_test,ystar_var,(models)
    else:
        return ystar_test,ystar_var

In [ ]:
def run_GPyTorchMO(xtrain,ytrain,xtest,alpha,strain=[],stest=[],urtrain=[],urtest=[],
           fitlinear=False,return_models=False,ard=False,mo_ard=False,initPhis=None,hasTime=False):
    if fitlinear:
        # fit linear model
        linearmodel = ElasticNet(alpha = alpha)
        linearmodel.fit(xtrain,ytrain)
        # get predictions for training data and test data using the linear model
        y_hat_train = linearmodel.predict(xtrain)
        y_hat_test = linearmodel.predict(xtest)
        # get the residual terms for output
        res_train = ytrain - y_hat_train
    else:
        res_train = ytrain
    ystar_test = np.zeros([xtest.shape[0],ytrain.shape[1]])
    ystar_var = np.zeros([xtest.shape[0],ytrain.shape[1]])
    
    # prepare the training and test data with one output column 
    # and one additional column in input indicating the output index
    xtrain_aug = np.hstack([xtrain,strain,urtrain])
    xtest_aug = np.hstack([xtest,stest,urtest])
    xtrain_f,res_train_f = flattenMO(xtrain_aug,res_train.shape[1],res_train)
    xtest_f = flattenMO(xtest_aug,res_train.shape[1])
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    xtrain_f_tensor = torch.tensor(np.array(xtrain_f,dtype='float32'))
    ytrain_f_tensor = torch.tensor(np.array(res_train_f,dtype='float32')).flatten()
    xtest_f_tensor = torch.tensor(np.array(xtest_f,dtype='float32'))
    
    model = ExactGPMOModel(xtrain_f_tensor, 
                         ytrain_f_tensor, 
                         likelihood, ytrain.shape[1], 
                         ard=ard,mo_ard=mo_ard,initPhis=initPhis,hasTime=hasTime)
    train_GPyTorch(model,likelihood,xtrain_f_tensor,ytrain_f_tensor,iterations=100,verbose=False)
    # get predictions
    model.eval()
    likelihood.eval()
    _rstar = likelihood(model(xtest_f_tensor))
    ystar_f_test = _rstar.mean.detach().numpy().ravel()
    ystar_f_var = _rstar.variance.detach().numpy().ravel()
    # unflatten ystar_f_test
    ystar_test = unflattenMO(ystar_f_test,ytrain.shape[1])
    ystar_var = unflattenMO(ystar_f_var,ytrain.shape[1])


    if fitlinear:
        ystar_test = y_hat_test + ystar_test

    if return_models:
        if fitlinear:
            return ystar_test,ystar_var,(model,linearmodel)
        else:
            return ystar_test,ystar_var,(model)
    else:
        return ystar_test,ystar_var

In [ ]:
def flattenMO(X,ny,y=[]):
    Xf = np.vstack([X]*ny)
    I = np.repeat(np.arange(0,ny),X.shape[0]).reshape(-1,1)
    Xf = np.hstack([Xf,I])
    if len(y) > 0:
        yf = y.reshape(-1,1,order='F')
        return Xf,yf
    else:
        return Xf

In [ ]:
def unflattenMO(y,ny):
    return y.reshape(int(y.shape[0]/ny),ny,order='F')


In [ ]:
class MOKernel(gpytorch.kernels.Kernel):
    is_stationary = False
    has_lengthscale = False

    # We will register the parameter when initializing the kernel
    def __init__(self,
                 phis_constraint=None,
                 taus_constraint=None,
                 num_tasks=4,
                 ard=False,
                 **kwargs):
        super().__init__(**kwargs)
        
        # register parameters
        # taus
        if ard:
            num_ard_dims = num_tasks
        else:
            num_ard_dims = 1
        self.register_parameter(name='raw_taus',
                                parameter=torch.nn.Parameter(
                                    torch.ones(*self.batch_shape,1,num_ard_dims))
                               )
        if taus_constraint is None:
            taus_constraint = Positive()
        self.register_constraint('raw_taus',taus_constraint)
        # NOTE - not setting any prior on the taus parameter yet
        # phis
        self.register_parameter(name='raw_phis',
                                parameter=torch.nn.Parameter(
                                    torch.ones(*self.batch_shape,
                                                1,int(0.5*num_tasks*(num_tasks-1)))
                                )
                               )
        if phis_constraint is None:
            # contraint the angles to be between -pi and +pi
            #phis_constraint = Interval(-3.141,3.141)
            phis_constraint = Interval(0,3.141)
        self.register_constraint('raw_phis',phis_constraint)
        # NOTE - not setting any prior on the phis parameter yet
        self.num_tasks = num_tasks
                
    @property
    def phis(self):
        return self.raw_phis_constraint.transform(self.raw_phis)
    @phis.setter
    def phis(self,value):
        return self._set_phis(value)
    
    def _set_phis(self,value):
        if not torch.is_tensor(value):
            value = torch.as_tensor(value).to(self.raw_phis)
        self.initialize(raw_phis=self.raw_phis_constraint.inverse_transform(value))
        
    @property
    def taus(self):
        return self.raw_taus_constraint.transform(self.raw_taus)
    @taus.setter
    def taus(self,value):
        return self._set_taus(value)
    
    def _set_taus(self,value):
        if not torch.is_tensor(value):
            value = torch.as_tensor(value).to(self.raw_taus)
        self.initialize(raw_taus=self.raw_taus_constraint.inverse_transform(value))
        

    # this is the kernel function
    def forward(self, x1, x2, **params):
        # construct the S and W matrices
        self.L = getSpherical(self.num_tasks,self.phis)
        if self.taus.shape[1] == 1:
            self.W = self.taus.flatten()*torch.matmul(self.L.t(),self.L)
        else:
            self.W = torch.matmul(torch.matmul(self.L.t(),self.L),
                              torch.diag(self.taus.flatten()))
        x1_int = x1.type(torch.LongTensor)
        x2_int = x2.type(torch.LongTensor)
        grid_x1, grid_x2 = torch.meshgrid(x1_int.flatten(), x2_int.flatten())
        inds = torch.vstack([grid_x1.flatten(),grid_x2.flatten()])
        Kmat = self.W[inds[0,:],inds[1:]].reshape(
            [x1.shape[0],x2.shape[0]])
        return Kmat

In [ ]:
def train_GPyTorch(model,likelihood,train_x,train_y,iterations=100,verbose=False):
    model = model.train()
    likelihood = likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    for i in range(iterations):
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)

        loss.backward()
        if verbose:
            print('Iter %d/%d - Loss: %.3f' % (i + 1, iterations, loss.item()))
        optimizer.step()
    if verbose:
        print('Optimization done')

In [ ]:
def convert_rectangular_torch(inp):
    r = inp[0]
    sz = inp.shape[0]
    multi_sin = 1
    if sz == 1:
        convert=torch.zeros([2],dtype=inp.dtype,device=inp.device)
    else:
        convert=torch.zeros([sz],dtype=inp.dtype,device=inp.device)
    for i in range(1, sz-1):
        convert = convert.put_(torch.tensor([i-1],device=inp.device),r* multi_sin * torch.cos(inp[i]))
        multi_sin = multi_sin*torch.sin(inp[i])
    convert = convert.put_(torch.tensor([-2],device=inp.device),r* multi_sin *torch.cos(inp[-1]))
    convert = convert.put_(torch.tensor([-1],device=inp.device),r* multi_sin *torch.sin(inp[-1]))
    return convert

In [ ]:
def convert_spherical_torch(inp):
    result =[]

    for element in range(0, len(inp)):
        r = 0
        for i in range(0, len(inp[element])):
            r += inp[element][i]*inp[element][i]
        r = math.sqrt(r)
        convert = [r]
        for i in range (0 ,len(inp[element])-2):
            convert.append(round(math.acos(inp[element][i] / r),6))
            r = math.sqrt(r*r - inp[element][i]*inp[element][i])
        if(inp[element][-2] >= 0):
            convert.append(math.acos(inp[element][-2]/r))
        else:
            convert.append(2*math.pi - math.acos(inp[element][-2] /r))
        convert = torch.from_numpy(np.array(convert))
        result += [convert]
    result = torch.stack(result)
    return result

In [ ]:
def getSpherical(n_tasks,phis):

    phis_mat = torch.tril(torch.ones([n_tasks,n_tasks],dtype=torch.float32,device=phis.device))
    inds = torch.tril_indices(n_tasks-1,n_tasks-1)+1
    phis_mat = phis_mat.index_put_((inds[0,:],inds[1,:]),phis.flatten()).T
    # prepare the vectorized form of the output matrix
    vec = torch.ones([1,int(0.5*n_tasks*(n_tasks+1))],device=phis.device)
    for n in range(1,n_tasks):
        st = torch.sum(torch.arange(n+1))
        en = torch.sum(torch.arange(n+2))
        rng = torch.arange(st,en)
        vec = vec.index_put_((torch.zeros(rng.shape[0],dtype=torch.long),rng), 
                             convert_rectangular_torch(phis_mat[0:n+1,n]))

    S = torch.zeros([n_tasks,n_tasks],device=phis.device)
    inds = torch.tril_indices(n_tasks,n_tasks)
    S = S.index_put_((inds[0,:],inds[1,:]),vec.flatten()).T

    return S

In [ ]:
def getPhis(W):
    L = torch.cholesky(W).t()
    n_tasks = W.shape[1]
    vec = torch.ones([1,int(0.5*n_tasks*(n_tasks-1))],device=L.device)
    st = torch.tensor(0)
    for n in range(1,n_tasks):
        en = torch.sum(torch.arange(n+1))
        rng = torch.arange(st,en)
        rect = convert_spherical_torch(L[0:(n+1),n].reshape((1,n+1)))[0,1:].float()
        vec = vec.index_put_((torch.zeros(rng.shape[0],dtype=torch.long),rng), 
                             rect)
        st = en
    return vec

In [ ]:
def plotScatter(pred_means_df,pred_vars_df,year,target,pop,outputfilename=None):
    pred_means_df = pred_means_df.join(pd.DataFrame(pop))
    # aggregate at region+type level
    pred = pred_means_df.groupby(['region_TYPE']).mean()
    pred['pop{}'.format(year)] = pred_means_df.groupby(['region_TYPE'])['pop{}'.format(year)].sum()
    pred.name = 'Predicted'
    vars_ = pred_vars_df.groupby(['region_TYPE']).mean()
    vars_.name = 'Predicted Variance'

    # Validating using DHS data
    df_dhs = pd.read_csv('dhs_data/{}_targets/targets_region.csv'.format(year))
    df_dhs.set_index('Unnamed: 0',inplace=True)
    result = df_dhs.join(pred,lsuffix='_true',rsuffix='_pred').join(vars_,rsuffix='_var')
    result['type'] = result.index.str[3:]
    
    x_ = result.loc[:,'{}_true'.format(target)].values
    y_ = result.loc[:,'{}_pred'.format(target)].values
    
    
    fig, ax = plt.subplots(figsize=(8,5))
    ax.set_xlabel('Survey-measured')
    ax.set_ylabel('Predicted')
    
    x2_ = sm.add_constant(x_) 
    est = sm.OLS(y_, x2_)
    est2 = est.fit()
    #print(est2.summary())
    
    sconst = 0.001
    # make scatter
    ax.scatter(result.loc[result['type']=='urban','{}_true'.format(target)],
           result.loc[result['type']=='urban','{}_pred'.format(target)],
           alpha=1,
           c='r',s=64)#,
           #s=sconst*(result.loc[result['type']=='urban','pop{}'.format(year)]))
    
    ax.scatter(result.loc[result['type']=='rural','{}_true'.format(target)],
           result.loc[result['type']=='rural','{}_pred'.format(target)],
           alpha=1,
           c='b',s=64)#,
           #s=sconst*(result.loc[result['type']=='urban','pop{}'.format(year)]))
    ax.set_xlim([-0.1,1])
    ax.set_ylim([-0.1,1])
    ax.legend(['Urban','Rural'],fontsize=20,loc="lower right",markerscale=1)#0.5),bbox_to_anchor=(1.35, 0))
    # add population legend
    #ps = []
    #ls = [50000,100000,500000,1000000]
    #for p in ls:
    #    ps.append(ax.scatter([], [], c='gray', alpha=0.8, s=p*sconst,label=str(p)))
    #leg = Legend(ax, ps,['50k','100k','500k','1000k'],fontsize=20,
    #             loc='center right', frameon=True, markerscale=1,
    #             title='Population\n({})'.format(year),labelspacing=1,bbox_to_anchor=(1.35, 0.5))
    #ax.add_artist(leg);

    #regression line
    x_test = np.linspace(0,1,10)#np.sort(x_)
    ax.plot(x_test, 
            LinearRegression().fit(
                x_[:,np.newaxis], y_[:,np.newaxis]
            ).predict(x_test[:,np.newaxis]),
            color='k'
           )
    ax.text(0.1,0.8, '$R^2$ = {:.2f}'.format(est2.rsquared), fontsize=20)


    plt.tight_layout()
    plt.show()
    if outputfilename != None:
        fig.savefig(outputfilename,dpi=192)
    return est2



In [ ]:
def prepPredictions(YPred,var,data_df,outputs,urtest):
    # prepare predictions for evaluation
    
    pred_means_df = pd.DataFrame(YPred,index=data_df.index,columns=outputs)
    pred_vars_df = pd.DataFrame(var,index=data_df.index,columns=outputs)

    pred_means_df = pred_means_df.join(pd.DataFrame(urtest.map({0:'rural',1:'urban'})))
    pred_means_df['region'] = pred_means_df.index.str[0:2]
    pred_means_df['region_TYPE'] = pred_means_df['region']+'_'+pred_means_df['TYPE']

    pred_vars_df = pred_vars_df.join(pd.DataFrame(urtest.map({0:'rural',1:'urban'})))
    pred_vars_df['region'] = pred_vars_df.index.str[0:2]
    pred_vars_df['region_TYPE'] = pred_vars_df['region']+'_'+pred_vars_df['TYPE']
    
    return pred_means_df,pred_vars_df

In [ ]:
def evaluateResults(pred_means_df,pred_vars_df,rmsevals,varvals,year,target):
    # evaluate at region+type level
    pred = pred_means_df.groupby(['region_TYPE']).mean()
    pred.name = 'Predicted'
    vars_ = pred_vars_df.groupby(['region_TYPE']).mean()
    vars_.name = 'Predicted Variance'

    # Validating using DHS data
    df_dhs = pd.read_csv('dhs_data/{}_targets/targets_region.csv'.format(year))
    df_dhs.set_index('Unnamed: 0',inplace=True)
    result = df_dhs.join(pred,lsuffix='_true',rsuffix='_pred').join(vars_,rsuffix='_var')
    result['type'] = result.index.str[3:]

    t = target
    tmp_df = result.loc[:,['{}_true'.format(t),'{}_pred'.format(t),'{}'.format(t),'type']]
    tmp_df['sqdiff'] = (tmp_df.loc[:,'{}_true'.format(t)] - tmp_df.loc[:,'{}_pred'.format(t)])**2
    
    r_ = np.sqrt(tmp_df.groupby('type')['sqdiff'].mean())
    p_ = pearsonr(tmp_df.loc[:,'{}_true'.format(t)],tmp_df.loc[:,'{}_pred'.format(t)])
    s_ = spearmanr(tmp_df.loc[:,'{}_true'.format(t)],tmp_df.loc[:,'{}_pred'.format(t)])
    v_ = tmp_df.groupby('type')[t].mean()
    
    rmsevals[t] = {'rmse':r_,'pearsonr':p_,'spearmanr':s_,'variance':v_}
    
    return rmsevals,varvals